In [2]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.dropout1 = nn.Dropout2d(0.25)  # Dropout after conv1

        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(128)
        self.dropout2 = nn.Dropout2d(0.2)  # Dropout after conv2
        
        self.conv3 = nn.Conv2d(128, 512, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(512)
        self.dropout3 = nn.Dropout2d(0.25)  # Dropout after conv3
        
        self.conv4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.dropout4 = nn.Dropout2d(0.25)  # Dropout after conv4
        
        # Fully connected layers
        self.fc1 = nn.Linear(512 * 3 * 3, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.dropout_fc1 = nn.Dropout(0.25)
        
        self.fc2 = nn.Linear(256, 512)
        self.bn_fc2 = nn.BatchNorm1d(512)
        self.dropout_fc2 = nn.Dropout(0.25)
        
        self.fc3 = nn.Linear(512, 7)  # 7 output classes (emotions)
        
        # Xavier initialization
        self._initialize_weights()

    def forward(self, x):
        # First Conv Layer
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)  # 48x48 -> 24x24
        x = self.dropout1(x)  # Apply dropout
        
        # Second Conv Layer
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)  # 24x24 -> 12x12
        x = self.dropout2(x)  # Apply dropout
        
        # Third Conv Layer
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)  # 12x12 -> 6x6
        x = self.dropout3(x)  # Apply dropout
        
        # Fourth Conv Layer
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)  # 6x6 -> 3x3
        x = self.dropout4(x)  # Apply dropout

        # Flatten
        x = x.view(x.size(0), -1)  # Flatten the tensor: 512 * 3 * 3
        
        # Fully connected layers
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout_fc1(x)  # Apply dropout
        
        x = F.relu(self.bn_fc2(self.fc2(x)))
        x = self.dropout_fc2(x)  # Apply dropout
        
        x = self.fc3(x)  # Output layer
        
        return x, F.log_softmax(x, dim=1)  # Raw logits, Log-Softmax for normalized probabilities

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)


In [ ]:
import torch
import torch.nn as nn

# Recreate the model and load weights
model = SimpleCNN()
model.load_state_dict(torch.load("best_model.pth"))  # Load state dict
model.eval()  # Set to evaluation mode# Set the model to evaluation mode

# Define a dummy input with the correct input size
dummy_input = torch.randn(1, 1, 48, 48)  # Batch size=1, Channels=1, Height=48, Width=48

# Export the model to ONNX format
onnx_file_path = "./fer_cnn.onnx"
torch.onnx.export(
    model,                           # The PyTorch model
    dummy_input,                     # Example input
    onnx_file_path,                  # File path to save the ONNX model
    input_names=["input"],           # Optional: Name the input tensor
    output_names=["output"],         # Optional: Name the output tensor
    dynamic_axes={                   # Optional: Enable dynamic axes
        "input": {0: "batch_size"},  # Dynamic batch size for input
        "output": {0: "batch_size"}  # Dynamic batch size for output
    },
    opset_version=11                 # ONNX opset version
)

print(f"Model has been exported to {onnx_file_path}")


/tmp/ipykernel_2478695/3603453630.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))  # Load state dict


Model has been exported to ./fer_cnn.onnx


In [1]:
# load libraries
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
from PIL import Image

# download model
model_path = "model.pt"

# load model
model = YOLO(model_path)

C:\Users\minnb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Export the model to ONNX format
model.export(format="onnx")

print("Model has been exported to ONNX format.")


Ultralytics 8.3.38  Python-3.11.9 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i7-12650H)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'model.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...
   --------------------------------------- 279.7/279.7 MB 38.6 MB/s eta 0:00:00

requirements: AutoUpdate success  19.3s, installed 1 package: ['onnxruntime-gpu']
requirements:  Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.13.1 opset 17...
ONNX: slimming with onnxslim 0.1.42...
ONNX: simplifier failure: FLOAT8E4M3FN
ONNX: export success  20.9s, saved as 'model.onnx' (11.7 MB)

Export complete (21.4s)
Results saved to C:\Studies\AIT\3rd semester\CV\CV-Project-Emotion-Detection\Model
Predict:         yolo predict task=detect model=model.onnx imgsz=640  
Validate

In [7]:
model.names

{0: 'FACE'}